# 🎯 Churn Prediction with XGBoost

**Objective**: Maximize Balanced Accuracy

**Key considerations**:
- No data leakage: removed `days_since_last_activity` (consequence of churn, not cause)
- No `cancel` page features (cancel = churn confirmation)
- Activity patterns instead of recency features
- XGBoost with threshold optimization for balanced accuracy

## 1. Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import (
    balanced_accuracy_score, roc_auc_score, f1_score, 
    confusion_matrix, classification_report, roc_curve
)

import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')

# Constants
REFERENCE_DATE = pd.to_datetime('2018-11-20')
RANDOM_STATE = 42

print("✅ Setup complete")

## 2. Load Data

In [ ]:
# Load training data
df_train = pd.read_csv('df_train_sample.csv')
df_train['time'] = pd.to_datetime(df_train['time'])
df_train['registration'] = pd.to_datetime(df_train['registration'])

print(f"📊 Training data: {len(df_train):,} events")
print(f"👥 Unique users: {df_train['userId'].nunique():,}")
print(f"🎯 Churn rate: {df_train.groupby('userId')['will_churn_10days'].first().mean()*100:.2f}%")

## 3. Feature Engineering

### Key design decisions:
1. **No `days_since_last_activity`**: This leaks information because churners naturally have longer inactivity (consequence, not cause)
2. **No `cancel` page features**: Cancel confirmation = churn, so it's direct leakage
3. **Activity patterns**: Use average gaps, consistency scores, and regularity metrics instead of recency

In [ ]:
def create_features(df, reference_date):
    """
    Create user-level features for churn prediction.
    
    Args:
        df: Event-level DataFrame
        reference_date: Date to calculate features up to
    
    Returns:
        DataFrame with one row per user and all features
    """
    
    # Filter events up to reference date
    df_filtered = df[df['time'] <= reference_date].copy()
    
    # Initialize user DataFrame
    user_features = pd.DataFrame()
    user_features['userId'] = df_filtered['userId'].unique()
    
    # =========================================================================
    # BASIC USER INFO
    # =========================================================================
    
    user_info = df_filtered.groupby('userId').agg({
        'gender': 'first',
        'level': 'last',  # Current subscription level
        'registration': 'first',
        'will_churn_10days': 'first'
    }).reset_index()
    
    user_features = user_features.merge(user_info, on='userId', how='left')
    
    # Encode categorical variables
    user_features['gender_encoded'] = LabelEncoder().fit_transform(
        user_features['gender'].fillna('unknown')
    )
    user_features['level_encoded'] = LabelEncoder().fit_transform(
        user_features['level'].fillna('unknown')
    )
    user_features['is_paid'] = (user_features['level'] == 'paid').astype(int)
    
    # Account age
    user_features['days_since_registration'] = (
        reference_date - user_features['registration']
    ).dt.days
    
    # =========================================================================
    # GLOBAL ACTIVITY METRICS
    # =========================================================================
    
    activity = df_filtered.groupby('userId').agg({
        'ts': 'count',
        'sessionId': 'nunique',
        'time': ['min', 'max'],
        'length': ['sum', 'mean', 'count'],
    })
    activity.columns = [
        'total_events', 'unique_sessions', 
        'first_activity', 'last_activity',
        'total_listening_time', 'avg_song_length', 'songs_with_length'
    ]
    activity = activity.reset_index()
    user_features = user_features.merge(activity, on='userId', how='left')
    
    # Activity span (how long the user has been active)
    user_features['activity_span_days'] = (
        pd.to_datetime(user_features['last_activity']) - 
        pd.to_datetime(user_features['first_activity'])
    ).dt.days + 1
    
    # Average activity rates
    user_features['events_per_day'] = (
        user_features['total_events'] / user_features['activity_span_days']
    ).replace([np.inf, -np.inf], 0).fillna(0)
    
    user_features['sessions_per_day'] = (
        user_features['unique_sessions'] / user_features['activity_span_days']
    ).replace([np.inf, -np.inf], 0).fillna(0)
    
    user_features['events_per_session'] = (
        user_features['total_events'] / user_features['unique_sessions']
    ).replace([np.inf, -np.inf], 0).fillna(0)
    
    user_features['listening_time_per_session'] = (
        user_features['total_listening_time'] / user_features['unique_sessions']
    ).replace([np.inf, -np.inf], 0).fillna(0)
    
    # =========================================================================
    # ACTIVITY PATTERN FEATURES (replacing days_since_last_activity)
    # These capture user behavior patterns without leaking churn information
    # =========================================================================
    
    def calculate_activity_patterns(group):
        """Calculate session gap patterns for a user."""
        sessions = group.groupby('sessionId')['time'].min().sort_values()
        
        if len(sessions) < 2:
            return pd.Series({
                'avg_gap_between_sessions': 0,
                'std_gap_between_sessions': 0,
                'max_gap_between_sessions': 0,
                'min_gap_between_sessions': 0,
                'median_gap_between_sessions': 0,
            })
        
        # Calculate gaps between consecutive sessions (in days)
        gaps = sessions.diff().dropna().dt.total_seconds() / (24 * 3600)
        
        return pd.Series({
            'avg_gap_between_sessions': gaps.mean(),
            'std_gap_between_sessions': gaps.std() if len(gaps) > 1 else 0,
            'max_gap_between_sessions': gaps.max(),
            'min_gap_between_sessions': gaps.min(),
            'median_gap_between_sessions': gaps.median(),
        })
    
    activity_patterns = df_filtered.groupby('userId').apply(
        calculate_activity_patterns
    ).reset_index()
    user_features = user_features.merge(activity_patterns, on='userId', how='left')
    
    # Consistency score: lower std/mean ratio = more consistent user
    user_features['consistency_score'] = 1 / (
        1 + user_features['std_gap_between_sessions'] / 
        (user_features['avg_gap_between_sessions'] + 0.1)
    )
    
    # Gap variability (coefficient of variation)
    user_features['gap_variability'] = (
        user_features['std_gap_between_sessions'] / 
        (user_features['avg_gap_between_sessions'] + 0.1)
    )
    
    # =========================================================================
    # PAGE INTERACTIONS (excluding 'Cancel' - it's leakage)
    # =========================================================================
    
    # Pages to track (NO CANCEL)
    pages_to_track = [
        'NextSong', 'Home', 'Thumbs Up', 'Thumbs Down', 
        'Add to Playlist', 'Add Friend', 'Roll Advert', 
        'Downgrade', 'Submit Downgrade', 'Upgrade',
        'Error', 'Help', 'Settings', 'Logout'
    ]
    
    page_counts = df_filtered.groupby(['userId', 'page']).size().unstack(fill_value=0)
    
    for page in pages_to_track:
        col_name = f'page_{page.lower().replace(" ", "_")}'
        if page in page_counts.columns:
            temp = page_counts[[page]].reset_index()
            temp.columns = ['userId', col_name]
            user_features = user_features.merge(temp, on='userId', how='left')
            user_features[col_name] = user_features[col_name].fillna(0)
        else:
            user_features[col_name] = 0
    
    # =========================================================================
    # ENGAGEMENT METRICS
    # =========================================================================
    
    # Positive interactions
    user_features['positive_interactions'] = (
        user_features['page_thumbs_up'] + 
        user_features['page_add_to_playlist'] + 
        user_features['page_add_friend']
    )
    
    # Negative interactions
    user_features['negative_interactions'] = (
        user_features['page_thumbs_down'] + 
        user_features['page_downgrade'] + 
        user_features['page_submit_downgrade']
    )
    
    # Interaction rates
    user_features['positive_interaction_rate'] = (
        user_features['positive_interactions'] / user_features['total_events']
    ).fillna(0)
    
    user_features['negative_interaction_rate'] = (
        user_features['negative_interactions'] / user_features['total_events']
    ).fillna(0)
    
    # Thumbs ratio (sentiment indicator)
    user_features['thumbs_ratio'] = (
        user_features['page_thumbs_down'] / 
        (user_features['page_thumbs_up'] + 1)
    )
    
    # Song listening ratio
    user_features['song_event_ratio'] = (
        user_features['page_nextsong'] / user_features['total_events']
    ).fillna(0)
    
    # Error rate
    user_features['error_rate'] = (
        user_features['page_error'] / user_features['total_events']
    ).fillna(0)
    
    # Help seeking rate
    user_features['help_rate'] = (
        user_features['page_help'] / user_features['total_events']
    ).fillna(0)
    
    # Churn signals (without cancel)
    user_features['churn_signals'] = (
        user_features['page_downgrade'] + 
        user_features['page_submit_downgrade']
    )
    
    # =========================================================================
    # TEMPORAL FEATURES (multi-period activity)
    # =========================================================================
    
    periods = [3, 7, 14, 21]
    
    for days in periods:
        period_start = reference_date - pd.Timedelta(days=days)
        period_data = df_filtered[df_filtered['time'] >= period_start]
        
        # Events in period
        period_events = period_data.groupby('userId').size().reset_index()
        period_events.columns = ['userId', f'events_last_{days}d']
        user_features = user_features.merge(period_events, on='userId', how='left')
        user_features[f'events_last_{days}d'] = user_features[f'events_last_{days}d'].fillna(0)
        
        # Sessions in period
        period_sessions = period_data.groupby('userId')['sessionId'].nunique().reset_index()
        period_sessions.columns = ['userId', f'sessions_last_{days}d']
        user_features = user_features.merge(period_sessions, on='userId', how='left')
        user_features[f'sessions_last_{days}d'] = user_features[f'sessions_last_{days}d'].fillna(0)
        
        # Thumbs down in period
        period_thumbs_down = period_data[period_data['page'] == 'Thumbs Down'].groupby('userId').size().reset_index()
        period_thumbs_down.columns = ['userId', f'thumbs_down_last_{days}d']
        user_features = user_features.merge(period_thumbs_down, on='userId', how='left')
        user_features[f'thumbs_down_last_{days}d'] = user_features[f'thumbs_down_last_{days}d'].fillna(0)
    
    # Activity trends (comparing recent vs older activity)
    user_features['trend_7d_vs_14d'] = (
        user_features['events_last_7d'] / 
        (user_features['events_last_14d'] - user_features['events_last_7d'] + 1)
    ).replace([np.inf, -np.inf], 0).clip(-10, 10)
    
    user_features['trend_3d_vs_7d'] = (
        user_features['events_last_3d'] / 
        (user_features['events_last_7d'] - user_features['events_last_3d'] + 1)
    ).replace([np.inf, -np.inf], 0).clip(-10, 10)
    
    # Activity acceleration (is decline speeding up?)
    user_features['activity_acceleration'] = (
        user_features['trend_3d_vs_7d'] - user_features['trend_7d_vs_14d']
    )
    
    # Recent activity ratio
    user_features['recent_activity_ratio'] = (
        user_features['events_last_7d'] / (user_features['total_events'] + 1)
    )
    
    # =========================================================================
    # NEGATIVE ACTIONS PATTERNS
    # =========================================================================
    
    # Average negative actions per day (overall)
    user_features['avg_negative_actions_per_day'] = (
        user_features['negative_interactions'] / user_features['activity_span_days']
    ).replace([np.inf, -np.inf], 0).fillna(0)
    
    # Negative actions last 7d compared to average
    # First calculate negative actions in last 7 days
    last_7d_start = reference_date - pd.Timedelta(days=7)
    negative_pages = ['Thumbs Down', 'Downgrade', 'Submit Downgrade']
    negative_last_7d = df_filtered[
        (df_filtered['time'] >= last_7d_start) & 
        (df_filtered['page'].isin(negative_pages))
    ].groupby('userId').size().reset_index()
    negative_last_7d.columns = ['userId', 'negative_actions_last_7d']
    user_features = user_features.merge(negative_last_7d, on='userId', how='left')
    user_features['negative_actions_last_7d'] = user_features['negative_actions_last_7d'].fillna(0)
    
    # Ratio: recent negative actions vs average
    user_features['negative_actions_last7d_vs_avg'] = (
        (user_features['negative_actions_last_7d'] / 7) / 
        (user_features['avg_negative_actions_per_day'] + 0.01)
    ).replace([np.inf, -np.inf], 0).clip(0, 100)
    
    # =========================================================================
    # THUMBS UP RECENCY (without leakage)
    # =========================================================================
    
    # Days since last thumbs up (calculated from activity span, not reference date)
    thumbs_up_events = df_filtered[df_filtered['page'] == 'Thumbs Up']
    last_thumbs_up = thumbs_up_events.groupby('userId')['time'].max().reset_index()
    last_thumbs_up.columns = ['userId', 'last_thumbs_up_time']
    user_features = user_features.merge(last_thumbs_up, on='userId', how='left')
    
    # Days between last thumbs up and last activity (not reference date - avoids leakage)
    user_features['days_without_thumbs_up'] = (
        pd.to_datetime(user_features['last_activity']) - 
        pd.to_datetime(user_features['last_thumbs_up_time'])
    ).dt.days
    
    # For users who never gave thumbs up, use activity span
    user_features['days_without_thumbs_up'] = user_features['days_without_thumbs_up'].fillna(
        user_features['activity_span_days']
    )
    
    # Normalize by activity span
    user_features['thumbs_up_recency_ratio'] = (
        user_features['days_without_thumbs_up'] / 
        (user_features['activity_span_days'] + 1)
    ).clip(0, 1)
    
    # =========================================================================
    # BINARY RISK INDICATORS
    # =========================================================================
    
    user_features['has_downgraded'] = (user_features['page_downgrade'] > 0).astype(int)
    user_features['has_errors'] = (user_features['page_error'] > 0).astype(int)
    user_features['has_sought_help'] = (user_features['page_help'] > 0).astype(int)
    user_features['is_low_engagement'] = (user_features['positive_interactions'] < 1).astype(int)
    user_features['is_declining'] = (user_features['trend_7d_vs_14d'] < 0.5).astype(int)
    user_features['has_negative_sentiment'] = (user_features['thumbs_ratio'] > 0.5).astype(int)
    user_features['is_inactive_7d'] = (user_features['events_last_7d'] == 0).astype(int)
    user_features['is_inactive_14d'] = (user_features['events_last_14d'] == 0).astype(int)
    
    # =========================================================================
    # COMPOSITE SCORES
    # =========================================================================
    
    # Frustration score
    user_features['frustration_score'] = (
        user_features['thumbs_ratio'] * 2 +
        user_features['help_rate'] * 3 +
        user_features['error_rate'] * 2 +
        user_features['has_downgraded'] * 5
    )
    
    # Engagement score (higher = more engaged)
    user_features['engagement_score'] = (
        user_features['positive_interaction_rate'] * 3 +
        user_features['song_event_ratio'] * 2 +
        user_features['consistency_score'] * 2 +
        (1 - user_features['thumbs_up_recency_ratio']) * 1
    )
    
    # Risk score
    user_features['risk_score'] = (
        user_features['frustration_score'] +
        user_features['negative_actions_last7d_vs_avg'] * 2 +
        user_features['is_declining'] * 3 +
        user_features['is_inactive_7d'] * 2
    )
    
    # =========================================================================
    # CLEANUP
    # =========================================================================
    
    # Drop temporary columns
    cols_to_drop = [
        'registration', 'first_activity', 'last_activity', 
        'gender', 'level', 'last_thumbs_up_time'
    ]
    user_features = user_features.drop(columns=[c for c in cols_to_drop if c in user_features.columns])
    
    # Fill any remaining NaN
    numeric_cols = user_features.select_dtypes(include=[np.number]).columns
    user_features[numeric_cols] = user_features[numeric_cols].fillna(0)
    
    return user_features

In [ ]:
# Create features
user_df = create_features(df_train, REFERENCE_DATE)

print(f"✅ Features created: {user_df.shape}")
print(f"\n📊 Feature columns ({len(user_df.columns) - 2} features + userId + target):")
print(user_df.columns.tolist())

In [ ]:
# Quick data check
print("📊 Target distribution:")
print(user_df['will_churn_10days'].value_counts())
print(f"\nChurn rate: {user_df['will_churn_10days'].mean()*100:.2f}%")

## 4. Prepare Data for Modeling

In [ ]:
# Define feature columns (exclude userId and target)
exclude_cols = ['userId', 'will_churn_10days']
feature_cols = [col for col in user_df.columns if col not in exclude_cols]

X = user_df[feature_cols]
y = user_df['will_churn_10days']

print(f"📊 Features: {X.shape[1]}")
print(f"📊 Samples: {X.shape[0]}")

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print(f"\n🔹 Training set: {len(X_train)} samples")
print(f"🔹 Test set: {len(X_test)} samples")

## 5. XGBoost Model Training

In [ ]:
# Calculate scale_pos_weight for imbalanced data
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
print(f"⚖️ Scale pos weight: {scale_pos_weight:.2f}")

# XGBoost model
xgb_model = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.1,
    min_child_weight=3,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=RANDOM_STATE,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
cv_scores = cross_val_score(xgb_model, X_train, y_train, cv=cv, scoring='balanced_accuracy')

print(f"\n📊 Cross-validation Balanced Accuracy:")
print(f"   Mean: {cv_scores.mean():.4f}")
print(f"   Std:  {cv_scores.std():.4f}")
print(f"   Scores: {cv_scores.round(4)}")

In [ ]:
# Train final model
xgb_model.fit(X_train, y_train)

# Predictions
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]
y_pred = xgb_model.predict(X_test)

# Evaluate with default threshold (0.5)
print("📊 Results with default threshold (0.5):")
print(f"   Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred):.4f}")
print(f"   ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")
print(f"   F1 Score: {f1_score(y_test, y_pred):.4f}")

## 6. Threshold Optimization for Balanced Accuracy

In [ ]:
def find_optimal_threshold(y_true, y_proba, metric='balanced_accuracy'):
    """
    Find the optimal threshold to maximize balanced accuracy.
    """
    thresholds = np.arange(0.1, 0.9, 0.01)
    scores = []
    
    for thresh in thresholds:
        y_pred = (y_proba >= thresh).astype(int)
        score = balanced_accuracy_score(y_true, y_pred)
        scores.append(score)
    
    best_idx = np.argmax(scores)
    best_threshold = thresholds[best_idx]
    best_score = scores[best_idx]
    
    return best_threshold, best_score, thresholds, scores

# Find optimal threshold
best_thresh, best_score, thresholds, scores = find_optimal_threshold(y_test, y_pred_proba)

print(f"🎯 Optimal threshold: {best_thresh:.2f}")
print(f"🎯 Best Balanced Accuracy: {best_score:.4f}")

In [ ]:
# Visualize threshold optimization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Balanced accuracy vs threshold
ax1 = axes[0]
ax1.plot(thresholds, scores, 'b-', linewidth=2)
ax1.axvline(x=best_thresh, color='red', linestyle='--', label=f'Optimal: {best_thresh:.2f}')
ax1.axvline(x=0.5, color='gray', linestyle=':', label='Default: 0.50')
ax1.scatter([best_thresh], [best_score], color='red', s=100, zorder=5)
ax1.set_xlabel('Threshold', fontsize=12)
ax1.set_ylabel('Balanced Accuracy', fontsize=12)
ax1.set_title('Threshold Optimization', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Confusion matrix with optimal threshold
ax2 = axes[1]
y_pred_optimal = (y_pred_proba >= best_thresh).astype(int)
cm = confusion_matrix(y_test, y_pred_optimal)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax2,
            xticklabels=['No Churn', 'Churn'],
            yticklabels=['No Churn', 'Churn'])
ax2.set_xlabel('Predicted', fontsize=12)
ax2.set_ylabel('Actual', fontsize=12)
ax2.set_title(f'Confusion Matrix (threshold={best_thresh:.2f})', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('threshold_optimization.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Final evaluation with optimal threshold
y_pred_optimal = (y_pred_proba >= best_thresh).astype(int)

print("="*60)
print("📊 FINAL RESULTS (Optimal Threshold)")
print("="*60)
print(f"\n🎯 Threshold: {best_thresh:.2f}")
print(f"\n📈 Metrics:")
print(f"   Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_optimal):.4f}")
print(f"   ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")
print(f"   F1 Score: {f1_score(y_test, y_pred_optimal):.4f}")

print(f"\n📋 Classification Report:")
print(classification_report(y_test, y_pred_optimal, target_names=['No Churn', 'Churn']))

## 7. Feature Importance

In [ ]:
# Feature importance
importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

print("📊 Top 20 Most Important Features:")
print(importance_df.head(20).to_string(index=False))

In [ ]:
# Visualize feature importance
fig, ax = plt.subplots(figsize=(10, 10))

top_20 = importance_df.head(20)
colors = plt.cm.viridis(np.linspace(0.8, 0.2, len(top_20)))

ax.barh(range(len(top_20)), top_20['importance'], color=colors)
ax.set_yticks(range(len(top_20)))
ax.set_yticklabels(top_20['feature'])
ax.invert_yaxis()
ax.set_xlabel('Importance', fontsize=12)
ax.set_title('Top 20 Feature Importances (XGBoost)', fontsize=14, fontweight='bold')
ax.grid(True, axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('feature_importance_xgb.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Train Final Model on All Data

In [ ]:
# Train on all data for submission
print("🎯 Training final model on all data...")

final_model = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.1,
    min_child_weight=3,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=RANDOM_STATE,
    use_label_encoder=False,
    eval_metric='logloss'
)

final_model.fit(X, y)

# Store optimal threshold
OPTIMAL_THRESHOLD = best_thresh

print(f"✅ Final model trained on {len(X)} samples")
print(f"✅ Optimal threshold: {OPTIMAL_THRESHOLD:.2f}")

---

# 📤 9. Kaggle Submission

Generate predictions for the test dataset.

In [ ]:
def create_submission(df_test, reference_date, model, feature_cols, threshold=0.5, output_file='submission.csv'):
    """
    Generate Kaggle submission file.
    
    Args:
        df_test: Test event-level DataFrame
        reference_date: Reference date for feature calculation
        model: Trained model
        feature_cols: List of feature columns
        threshold: Classification threshold
        output_file: Output filename
    
    Returns:
        Submission DataFrame
    """
    print("🔄 Creating features for test data...")
    
    # Create features (reuse the same function)
    test_features = create_features(df_test, reference_date)
    
    # Ensure all required columns exist
    for col in feature_cols:
        if col not in test_features.columns:
            test_features[col] = 0
    
    # Prepare X
    X_test = test_features[feature_cols]
    
    # Predictions
    print("🎯 Generating predictions...")
    probas = model.predict_proba(X_test)[:, 1]
    predictions = (probas >= threshold).astype(int)
    
    # Create submission file
    submission = pd.DataFrame({
        'id': test_features['userId'].astype(int),
        'target': predictions
    })
    
    # Save
    submission.to_csv(output_file, index=False)
    
    print(f"\n✅ Submission saved: {output_file}")
    print(f"📊 Statistics:")
    print(f"   Total users: {len(submission):,}")
    print(f"   Predicted churn: {submission['target'].sum():,} ({submission['target'].mean()*100:.1f}%)")
    print(f"   Predicted no-churn: {(submission['target']==0).sum():,}")
    
    return submission, probas

In [ ]:
# ============================================================
# 🎯 GENERATE KAGGLE SUBMISSION
# ============================================================

# Load test data
df_test = pd.read_parquet('df_test.parquet')
df_test['time'] = pd.to_datetime(df_test['time'])
df_test['registration'] = pd.to_datetime(df_test['registration'])

print(f"📊 Test data loaded: {len(df_test):,} events")
print(f"👥 Unique users: {df_test['userId'].nunique():,}")

In [ ]:
# Generate submission with optimal threshold
submission, test_probas = create_submission(
    df_test=df_test,
    reference_date=REFERENCE_DATE,
    model=final_model,
    feature_cols=feature_cols,
    threshold=OPTIMAL_THRESHOLD,
    output_file='submission.csv'
)

In [ ]:
# Preview submission
print("\n📋 Submission preview:")
submission.head(10)

In [ ]:
# Visualize prediction distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Probability distribution
ax1 = axes[0]
ax1.hist(test_probas, bins=50, color='#3498db', edgecolor='black', alpha=0.7)
ax1.axvline(x=OPTIMAL_THRESHOLD, color='red', linestyle='--', 
            linewidth=2, label=f'Threshold: {OPTIMAL_THRESHOLD:.2f}')
ax1.set_xlabel('Churn Probability', fontsize=12)
ax1.set_ylabel('Number of Users', fontsize=12)
ax1.set_title('Prediction Probability Distribution', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Prediction counts
ax2 = axes[1]
counts = submission['target'].value_counts().sort_index()
bars = ax2.bar(['No Churn (0)', 'Churn (1)'], counts.values, 
               color=['#2ecc71', '#e74c3c'], edgecolor='black')
ax2.set_ylabel('Number of Users', fontsize=12)
ax2.set_title('Prediction Distribution', fontsize=14, fontweight='bold')

for bar, count in zip(bars, counts.values):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 20, 
             f'{count:,}\n({count/len(submission)*100:.1f}%)', 
             ha='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('submission_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

---

## 📊 Summary

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════════╗
║                         PIPELINE SUMMARY                             ║
╚══════════════════════════════════════════════════════════════════════╝
""")

print(f"""
📊 DATA
{'─'*50}
• Training events: {len(df_train):,}
• Training users: {user_df.shape[0]:,}
• Features: {len(feature_cols)}
• Churn rate: {y.mean()*100:.2f}%

🎯 MODEL
{'─'*50}
• Algorithm: XGBoost
• Optimal threshold: {OPTIMAL_THRESHOLD:.2f}
• CV Balanced Accuracy: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}
• Test Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_optimal):.4f}
• Test ROC-AUC: {roc_auc_score(y_test, y_pred_proba):.4f}

⚠️ LEAKAGE PREVENTION
{'─'*50}
• Removed: days_since_last_activity (consequence of churn)
• Removed: cancel page features (cancel = churn)
• Added: Activity pattern features (gaps, consistency)

📤 SUBMISSION
{'─'*50}
• File: submission.csv
• Users: {len(submission):,}
• Predicted churn rate: {submission['target'].mean()*100:.1f}%
""")

print("\n🔝 Top 10 Features:")
for i, row in importance_df.head(10).iterrows():
    print(f"   {row['feature']}: {row['importance']:.4f}")